In [1]:
import os
from sentence_transformers import SentenceTransformer

In [2]:
import dotenv
dotenv.load_dotenv()

True

In [3]:
model = SentenceTransformer('BAAI/bge-small-en-v1.5')

In [4]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [5]:
MONGODB_URL = os.getenv("MONGODB_URL")

In [6]:
# Create a new client and connect to the server
mongo_client = MongoClient(MONGODB_URL, server_api=ServerApi('1'))

In [18]:
question = "What are steps to take when finding projects to build your experience?"
question = "The percentage of the asset allocation with a duration longer than 7 years is less than 20% ?"

In [19]:
fileName = "SUNY RF - General Investment Policy and Guidelines.pdf"

In [20]:
question_embedding = model.encode(question, normalize_embeddings=True)

In [21]:
sum(question_embedding)

0.13422016796539538

```json
{
  "fields": [
    {
      "numDimensions": 384,
      "path": "embedding",
      "similarity": "dotProduct",
      "type": "vector"
    },
    {
      "path": "fileName",
      "type": "filter"
    }
  ]
}
```

In [22]:
query_vector = question_embedding.tolist()

In [23]:
results = mongo_client["RAG"]['DocumentEmbedding'].aggregate([
    {

        "$vectorSearch": {
            "index": "vector_index",
            "path": "embedding",
            "queryVector": query_vector,
            "numCandidates": 5,
            "limit": 5,
            "filter": {"fileName": {"$eq": fileName}}
        }

    },
    {

        '$project': {
            'embedding': 0,
            "_id": 0,
            "score": {"$meta": "vectorSearchScore"},
        }

    }

])

In [24]:
results = list(results)

In [25]:
results

[{'fileName': 'SUNY RF - General Investment Policy and Guidelines.pdf',
  'textIdx': 6,
  'pageLabel': '7',
  'text': 'Research  Foundation  for the State University of New York  \nInvestment Policy and Guidelines  \n \nSeptember 21, 2023  \n \n7 \n  \n \nD. Asset Allocation  \nIn order to  have a reasonable probability of consistently achieving the RF’s long term return objectives, \nthe Committee has adopted the asset allocation policies outlined below : \n \n \nOPERATIONAL POOL MEDIUM DURATION  \nAsset Class  Long Term Target %  Ranges  \nCash  3% 0-10% \n      \nFixed Income  31.5%  15-45% \nGovernment Bonds  12.5%    0-20%  \nCredit  19% 5-30% \n   \nEquities  33% 20-40% \nGlobal Equities  17% 10-30% \nHedged Equities  16% 0-30% \nPrivate Equity  0% 0% \n      \nAbsolute Return  15% 0-25% \n      \nReal Assets  17.5%  5-30% \nInflation -Protected  12.5%  0-20% \nCommodities  0% 0-5% \nGlobal Real Estate  5% 2-15% \n \n \n OPERATIONAL POOL  LONG DURATION  \nAsset Class  Long Term T